In [4]:
import numpy as np
import gensim, logging
import csv, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
import pandas as pd
import tensorflow as tf
import os
tf.executing_eagerly()

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
	try:
		# Currently, memory growth needs to be the same across GPUs
		for gpu in gpus:
			tf.config.experimental.set_memory_growth(gpu, True)
			logical_gpus = tf.config.experimental.list_logical_devices('GPU')
		print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
	except RuntimeError as e:
		# Memory growth must be set before GPUs have been initialized
		print(e)

import pickle
#with open("label_dict.pickle", 'rb') as f:
#    label_dict = pickle.load(f)
embedding_dim = 300
num_words = 300
all_labels = {"A01B","A01C","A01D","A01F","A01G","A01H","A01J","A01K","A01L","A01M","A01N","A21B","A21C","A21D","A22B","A22C","A23B","A23C","A23D","A23F","A23G","A23J","A23K","A23L","A23N","A23P","A23V","A23Y","A24B","A24C","A24D","A24F","A41B","A41C","A41D","A41F","A41G","A41H","A42B","A42C","A43B","A43C","A43D","A44B","A44C","A44D","A45B","A45C","A45D","A45F","A46B","A46D","A47B","A47C","A47D","A47F","A47G","A47H","A47J","A47K","A47L","A61B","A61C","A61D","A61F","A61G","A61H","A61J","A61K","A61L","A61M","A61N","A61P","A61Q","A62B","A62C","A62D","A63B","A63C","A63D","A63F","A63G","A63H","A63J","A63K","A99Z","B01B","B01D","B01F","B01J","B01L","B02B","B02C","B03B","B03C","B03D","B04B","B04C","B05B","B05C","B05D","B06B","B07B","B07C","B08B","B09B","B09C","B21B","B21C","B21D","B21F","B21G","B21H","B21J","B21K","B21L","B22C","B22D","B22F","B23B","B23C","B23D","B23F","B23G","B23H","B23K","B23P","B23Q","B24B","B24C","B24D","B25B","B25C","B25D","B25F","B25G","B25H","B25J","B26B","B26D","B26F","B27B","B27C","B27D","B27F","B27G","B27H","B27J","B27K","B27L","B27M","B27N","B28B","B28C","B28D","B29B","B29C","B29D","B29K","B29L","B30B","B31B","B31C","B31D","B31F","B32B","B33Y","B41B","B41C","B41D","B41F","B41G","B41J","B41K","B41L","B41M","B41N","B41P","B42B","B42C","B42D","B42F","B42P","B43K","B43L","B43M","B44B","B44C","B44D","B44F","B60B","B60C","B60D","B60F","B60G","B60H","B60J","B60K","B60L","B60M","B60N","B60P","B60Q","B60R","B60S","B60T","B60V","B60W","B60Y","B61B","B61C","B61D","B61F","B61G","B61H","B61J","B61K","B61L","B62B","B62C","B62D","B62H","B62J","B62K","B62L","B62M","B63B","B63C","B63G","B63H","B63J","B64B","B64C","B64D","B64F","B64G","B65B","B65C","B65D","B65F","B65G","B65H","B66B","B66C","B66D","B66F","B67B","B67C","B67D","B68B","B68C","B68F","B68G","B81B","B81C","B82B","B82Y","B99Z","C01B","C01C","C01D","C01F","C01G","C01P","C02F","C03B","C03C","C04B","C05B","C05C","C05D","C05F","C05G","C06B","C06C","C06D","C06F","C07B","C07C","C07D","C07F","C07G","C07H","C07J","C07K","C08B","C08C","C08F","C08G","C08H","C08J","C08K","C08L","C09B","C09C","C09D","C09F","C09G","C09H","C09J","C09K","C10B","C10C","C10F","C10G","C10H","C10J","C10K","C10L","C10M","C10N","C11B","C11C","C11D","C12C","C12F","C12G","C12H","C12J","C12L","C12M","C12N","C12P","C12Q","C12R","C12Y","C13B","C13K","C14B","C14C","C21B","C21C","C21D","C22B","C22C","C22F","C23C","C23D","C23F","C23G","C25B","C25C","C25D","C25F","C30B","C40B","C99Z","D01B","D01C","D01D","D01F","D01G","D01H","D02G","D02H","D02J","D03C","D03D","D03J","D04B","D04C","D04D","D04G","D04H","D05B","D05C","D05D","D06B","D06C","D06F","D06G","D06H","D06J","D06L","D06M","D06N","D06P","D06Q","D07B","D10B","D21B","D21C","D21D","D21F","D21G","D21H","D21J","D99Z","E01B","E01C","E01D","E01F","E01H","E02B","E02C","E02D","E02F","E03B","E03C","E03D","E03F","E04B","E04C","E04D","E04F","E04G","E04H","E05B","E05C","E05D","E05F","E05G","E05Y","E06B","E06C","E21B","E21C","E21D","E21F","E99Z","F01B","F01C","F01D","F01K","F01L","F01M","F01N","F01P","F02B","F02C","F02D","F02F","F02G","F02K","F02M","F02N","F02P","F03B","F03C","F03D","F03G","F03H","F04B","F04C","F04D","F04F","F05B","F05C","F05D","F15B","F15C","F15D","F16B","F16C","F16D","F16F","F16G","F16H","F16J","F16K","F16L","F16M","F16N","F16P","F16S","F16T","F17B","F17C","F17D","F21H","F21K","F21L","F21S","F21V","F21W","F21Y","F22B","F22D","F22G","F23B","F23C","F23D","F23G","F23H","F23J","F23K","F23L","F23M","F23N","F23Q","F23R","F24B","F24C","F24D","F24F","F24H","F24S","F24T","F24V","F25B","F25C","F25D","F25J","F26B","F27B","F27D","F27M","F28B","F28C","F28D","F28F","F28G","F41A","F41B","F41C","F41F","F41G","F41H","F41J","F42B","F42C","F42D","F99Z","G01B","G01C","G01D","G01F","G01G","G01H","G01J","G01K","G01L","G01M","G01N","G01P","G01Q","G01R","G01S","G01T","G01V","G01W","G02B","G02C","G02F","G03B","G03C","G03D","G03F","G03G","G03H","G04B","G04C","G04D","G04F","G04G","G04R","G05B","G05D","G05F","G05G","G06C","G06D","G06E","G06F","G06G","G06J","G06K","G06M","G06N","G06Q","G06T","G07B","G07C","G07D","G07F","G07G","G08B","G08C","G08G","G09B","G09C","G09D","G09F","G09G","G10B","G10C","G10D","G10F","G10G","G10H","G10K","G10L","G11B","G11C","G12B","G16B","G16C","G16H","G16Z","G21B","G21C","G21D","G21F","G21G","G21H","G21J","G21K","G99Z","H01B","H01C","H01F","H01G","H01H","H01J","H01K","H01L","H01M","H01P","H01Q","H01R","H01S","H01T","H02B","H02G","H02H","H02J","H02K","H02M","H02N","H02P","H02S","H03B","H03C","H03D","H03F","H03G","H03H","H03J","H03K","H03L","H03M","H04B","H04H","H04J","H04K","H04L","H04M","H04N","H04Q","H04R","H04S","H04T","H04W","H05B","H05C","H05F","H05G","H05H","H05K","H99Z","Y02A","Y02B","Y02C","Y02D","Y02E","Y02P","Y02T","Y02W","Y04S","Y10S","Y10T"}
def convert_all_labels(first_half):
	return tuple([ord(first_half[0])-65, int(first_half[1:3]), ord(first_half[3])-65])


import random
def convert_label_to_vector(label):
	first_half, second_half = label.split(" ")
	return tuple([ord(first_half[0])-65, int(first_half[1:3]), ord(first_half[3])-65])

wv = gensim.models.KeyedVectors.load("patents-small.wv", mmap='r')
porter = PorterStemmer()
ret = []
ret_labels = []
placeholder_vec = [0 for _ in range(embedding_dim)]


def get_sentence_vector(sentence):
	ret = []
	for word in sentence:
		if word.islower():
			try:
				ret.append(wv[porter.stem(word)])
			except:
				pass
	while len(ret) < num_words:
		ret.append(placeholder_vec)
	return np.vstack(ret[:num_words])
all_labels = {convert_all_labels(i) for i in all_labels}
with open("binary_label_dict-small.pickle", 'rb') as f:
    label_dict = pickle.load(f)

label_dict_keys = set(label_dict.keys())

input_lstm = tf.keras.Input(shape=(num_words, embedding_dim), name='input_1')
mask_lstm = tf.keras.layers.Masking(mask_value=0., input_shape=(num_words, embedding_dim))(input_lstm)
input_label = tf.keras.Input(shape=(embedding_dim), name='input_2')
patent_lstm = tf.keras.layers.LSTM(embedding_dim, input_shape=(num_words, embedding_dim), activation = 'tanh', recurrent_activation = 'sigmoid', recurrent_dropout = 0, unroll=False, use_bias=True
)(mask_lstm)
dense_patent_1 = tf.keras.layers.Dense(embedding_dim, activation='relu')(patent_lstm)
dense_label_1 =  tf.keras.layers.Dense(embedding_dim, activation='relu')(input_label)
dense_patent_2 = tf.keras.layers.Dense(embedding_dim, activation='relu')(dense_patent_1)
dense_label_2 =  tf.keras.layers.Dense(embedding_dim, activation='relu')(dense_label_1)
dense_patent_3 = tf.keras.layers.Dense(embedding_dim, activation='relu')(dense_patent_2)
dense_label_3 =  tf.keras.layers.Dense(embedding_dim, activation='relu')(dense_label_2)
dense_patent_4 = tf.keras.layers.Dropout(.1)(dense_patent_3)
dense_label_4=  tf.keras.layers.Dropout(.1)(dense_label_3)
concat = tf.keras.layers.Concatenate(axis=1)([dense_patent_4, dense_label_4])
dense_1 = tf.keras.layers.Dense( 2*embedding_dim, activation='relu')(concat)
output_binary = tf.keras.layers.Dense(1, activation="sigmoid", name='output_binary')(dense_1)


#lstm_enforce_1 = tf.keras.layers.Dense(200, activation='relu')(patent_lstm)
#lstm_enforce_2 = tf.keras.layers.Dense(1000, name='output_2')(lstm_enforce_1)
#model = tf.keras.Model(inputs={'input_1':input_lstm, 'input_2':input_label}, outputs={'output_1':output_binary, 'output_2':lstm_enforce_2})
model = tf.keras.Model(inputs={'input_1':input_lstm, 'input_2':input_label}, outputs=[output_binary])
try:
    model.load_weights('./lstmcheckpoint6.h5')
except:
    print('couldnt load')
model.compile(loss=['binary_crossentropy'],
			  optimizer='adam',
			  metrics=['accuracy', tf.keras.metrics.Precision()], experimental_run_tf_function=False)

#saver = tf.train.Saver(max_to_keep=4, keep_checkpoint_every_n_hours=2)

#model.compile(loss=['binary_crossentropy', 'binary_crossentropy'],
#			  optimizer='adam',
#			  metrics=['binary_accuracy'], loss_weights=[.75, .25], experimental_run_tf_function=False)
#test_loss, test_acc = model.evaluate(test_dataset, steps=1000)
label_len = len(label_dict_keys)
print(label_len)
left = [label_dict[tuple(i)] for i in label_dict_keys]
ns = []
predictions = []
label_vectors_vector = []
n = 0
ns = []
recalls = np.arange(0, 1.001, .01)
left_vectors = np.array(list(label_dict.keys()))
d=0
#for label, lstm_input_patent in zip(testing_arr_labels, testing_arr):
#	label = label[0]
for row in pd.read_csv('../test_dataset.csv',sep=',', header = None).to_numpy():
	label, description=row[0], row[1]
	label_vectors = [convert_label_to_vector(z.strip().strip('\'\"')) for z in label.split(',')]
	label_vectors = set([i for i in label_vectors if i in label_dict_keys])
	if label_vectors == set():
		continue
	lstm_input_patent = get_sentence_vector(word_tokenize(description.lower()))
    def cur_gen():
        for i in left:
            yield lstm_input_patent, i
	prediction = model.predict_generator(cur_gen)
	indices =np.flip(np.argsort(prediction, axis=0))
	#print(left_vectors[indices])
	#indices = np.argsort(prediction, axis=0)
	e = [1 if tuple(i[0]) in label_vectors else 0 for i in left_vectors[indices]]
	cur = []
	total_ones = e.count(1)
	n = 0
	ones_seen = 0
	while ones_seen < total_ones:
		if e[n] == 1:
			ones_seen = ones_seen+1
		n = n+1
		cur.append((ones_seen/float(n), ones_seen/float(total_ones)))
	new_cur = []
	for r in recalls:
		max_of_larger = max([i[0] for i in cur if i[1] >= r])
		new_cur.append(max_of_larger)
	ns.append(new_cur)
	print(max(new_cur))
	d = d+1
	#print(float(d)/7215)
	#print(np.mean(np.vstack(ns), axis=0)[0])
p = np.mean(np.array(ns), axis=0)
np.save("newest_ps.npy", p)
np.save("predictions.npy", predictions)
np.save("label_vectors_vector.npy", label_vectors_vector)
import matplotlib.pyplot as plt
plt.plot(recalls, p)
plt.ylim(0, 1)
plt.xlim(0, 1)
plt.margins(0.1)
#plt.xticks(range(len(distribution_dict.keys()))[::50], new_x[::50], rotation='vertical')
plt.ylabel('Precision', fontsize=16)
plt.xlabel('Recall', fontsize=16)
plt.tight_layout()
plt.show()
print(p) 
plt.savefig('binary_prc1.eps', format='eps')


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 119)

0.0016129032258064516
0.0017699115044247787
0.0015723270440251573
0.0016474464579901153
0.0015360983102918587
0.0015313935681470138
0.001594896331738437
0.0018050541516245488
0.1111111111111111
0.017241379310344827
0.015873015873015872
0.022222222222222223
0.06666666666666667
0.023809523809523808
0.0021929824561403508
0.045454545454545456
0.002976190476190476
0.037037037037037035
0.006521739130434782
0.005235602094240838
0.005698005698005698
0.1111111111111111
0.0034602076124567475
0.0036231884057971015
0.0032258064516129032
1.0
0.0032310177705977385
0.004032258064516129
0.02040816326530612
0.01020408163265306
0.0018115942028985507
0.003738317757009346
0.004081632653061225
0.1111111111111111
0.14285714285714285
0.09090909090909091
0.2
0.004016064257028112
0.02857142857142857
0.001718213058419244
0.012
0.3333333333333333
1.0
1.0
0.14285714285714285
0.5
0.021739130434782608
0.07692307692307693
0.02197802197802198
0.013513513513513514
0.020833333333333332
0.04827586206896552
0.02941176470

0.007481296758104738
0.0072992700729927005
0.00684931506849315
0.02142857142857143
0.008888888888888889
0.011235955056179775
0.01282051282051282
0.004728132387706856
0.014705882352941176
0.015625
0.01639344262295082
0.014705882352941176
0.023809523809523808
0.016
0.007462686567164179
0.00819672131147541
0.02127659574468085
0.019417475728155338
0.004807692307692308
0.010570824524312896
0.015151515151515152
0.008849557522123894
0.047619047619047616
0.018518518518518517
0.029411764705882353
0.023255813953488372
0.0057306590257879654
0.00975609756097561
0.008968609865470852
0.013513513513513514
0.017543859649122806
0.007692307692307693
0.006369426751592357
0.00847457627118644
0.01282051282051282
0.011235955056179775
0.015151515151515152
0.015384615384615385
0.014925373134328358
0.014084507042253521
0.0410958904109589
0.0125
0.016129032258064516
0.015151515151515152
0.017241379310344827
0.015625
0.01694915254237288
0.015384615384615385
0.011764705882352941
0.021739130434782608
0.01176470588

0.006172839506172839
0.008695652173913044
0.011363636363636364
0.008547008547008548
0.16666666666666666
0.013157894736842105
0.015384615384615385
0.00554016620498615
0.03125
0.01694915254237288
0.00510204081632653
0.015748031496062992
0.0036101083032490976
0.013157894736842105
0.007751937984496124
0.07692307692307693
0.034482758620689655
0.008928571428571428
0.0025188916876574307
0.005847953216374269
0.011235955056179775
0.1
0.010309278350515464
0.008064516129032258
0.008948545861297539
0.0064516129032258064
0.0021321961620469083
0.005076142131979695
0.01818181818181818
0.0019267822736030828
0.0026246719160104987
0.005361930294906166
0.0036363636363636364
0.015625
0.002638522427440633
0.009615384615384616
0.0031847133757961785
0.009852216748768473
0.004914004914004914
0.01282051282051282
0.01078167115902965
0.06451612903225806
0.00303951367781155
0.009345794392523364
0.015384615384615385
0.009433962264150943
0.004310344827586207
0.008733624454148471
0.0018552875695732839
0.018867924528

0.01152073732718894
0.030303030303030304
0.05263157894736842
0.05555555555555555
0.029411764705882353
0.1
0.0017152658662092624
0.015625
0.022988505747126436
0.010101010101010102
0.3333333333333333
0.16666666666666666
0.16279069767441862
0.25
0.2857142857142857
0.14285714285714285
0.012195121951219513
0.0027247956403269754
0.14285714285714285
0.02857142857142857
0.010033444816053512
0.010033444816053512
0.08333333333333333
0.3333333333333333
0.006944444444444444
0.023255813953488372
0.01056338028169014
0.012903225806451613
0.022727272727272728
0.023809523809523808
0.017391304347826087
0.041666666666666664
0.008438818565400843
0.00819672131147541
0.008771929824561403
0.006211180124223602
0.012048192771084338
0.011904761904761904
0.011235955056179775
0.1
0.003952569169960474
0.012345679012345678
0.08333333333333333
0.013513513513513514
0.023809523809523808
0.006896551724137931
0.010752688172043012
0.012345679012345678
0.006802721088435374
0.019021739130434784
0.005813953488372093
0.01904

0.005813953488372093
0.008333333333333333
0.03125
0.01818181818181818
0.020761245674740483
0.25
0.0273972602739726
0.1
0.023255813953488372
0.02631578947368421
0.07142857142857142
0.03773584905660377
0.058823529411764705
0.008771929824561403
0.02
0.007751937984496124
0.006802721088435374
0.016877637130801686
0.02631578947368421
0.005780346820809248
0.03571428571428571
0.05263157894736842
0.1111111111111111
0.0625
0.0425531914893617
0.011235955056179775
0.017543859649122806
0.09090909090909091
0.03571428571428571
0.018867924528301886
0.02564102564102564
0.2
0.03333333333333333
0.041666666666666664
0.25
0.00784313725490196
0.027932960893854747
0.058823529411764705
0.09090909090909091
0.012345679012345678
0.029411764705882353
0.024691358024691357
0.09090909090909091
0.1111111111111111
0.0392156862745098
0.05128205128205128
0.14285714285714285
0.08333333333333333
0.06666666666666667
0.16666666666666666
0.08333333333333333
0.011019283746556474
0.034482758620689655
0.03571428571428571
0.0277

...

0.05263157894736842
0.009746588693957114
0.07692307692307693
0.0594059405940594
0.0125
0.010958904109589041
0.011764705882352941
0.01953125
0.013888888888888888
0.013513513513513514
0.022222222222222223
0.03333333333333333
0.0273972602739726
0.02127659574468085
0.04
0.07142857142857142
0.1
0.02564102564102564
0.022222222222222223
0.007692307692307693
0.011111111111111112
0.012658227848101266
0.045454545454545456
0.045454545454545456
0.009523809523809525
0.023255813953488372
0.038461538461538464
0.023809523809523808
0.025
0.011904761904761904
0.025974025974025976
0.02702702702702703
0.029411764705882353
0.03125
0.06896551724137931
0.03225806451612903
0.010309278350515464
0.02127659574468085
0.013333333333333334
0.01694915254237288
0.025
0.045454545454545456
0.038461538461538464
0.012345679012345678
0.06666666666666667
0.0033003300330033004
0.023255813953488372
0.016129032258064516
0.030303030303030304
0.03225806451612903
0.02631578947368421
0.07692307692307693
0.011627906976744186
0.014

0.012195121951219513
0.0018832391713747645
0.0023094688221709007
0.0022988505747126436
0.017857142857142856
0.014925373134328358
0.02830188679245283
0.005847953216374269
0.045454545454545456
0.015384615384615385
0.0018050541516245488
0.009174311926605505
0.02127659574468085
0.03225806451612903
0.047619047619047616
0.002512562814070352
0.005780346820809248
0.009174311926605505
0.017857142857142856
0.0034602076124567475
0.05263157894736842
0.005747126436781609
0.003257328990228013
0.045454545454545456
0.006369426751592357
0.004464285714285714
0.019801980198019802
0.007246376811594203
0.002036659877800407
0.00477326968973747
0.008928571428571428
0.014925373134328358
0.008264462809917356
0.005
0.0028653295128939827
0.0034129692832764505
0.01694915254237288
0.002583979328165375
0.02608695652173913
0.007272727272727273
0.0040650406504065045
0.01818181818181818
0.01639344262295082
0.014705882352941176
0.0029069767441860465
0.005235602094240838
0.009708737864077669
0.005555555555555556
0.00781

0.0036363636363636364
0.019230769230769232
0.01639344262295082
0.04878048780487805
0.25
0.002551020408163265
0.007380073800738007
0.011037527593818985
0.05263157894736842
0.009523809523809525
0.05263157894736842
0.015267175572519083
0.0061162079510703364
0.002331002331002331
0.02564102564102564
0.007716049382716049
0.003067484662576687
0.008368200836820083
0.02666666666666667
0.02564102564102564
0.006607929515418502
0.02702702702702703
0.004629629629629629
0.012875536480686695
0.05263157894736842
0.02564102564102564
0.014705882352941176
0.022026431718061675
0.004338394793926247
0.0017985611510791368
0.006060606060606061
0.0625
0.017241379310344827
0.006956521739130435
0.020618556701030927
0.00909090909090909
0.012987012987012988
0.007009345794392523
0.010638297872340425
0.006756756756756757
0.010309278350515464
0.015384615384615385
0.01639344262295082
0.02564102564102564
0.017857142857142856
0.017391304347826087
0.01694915254237288
0.014705882352941176
0.014705882352941176
0.0108695652

0.00425531914893617
0.058823529411764705
0.02857142857142857
0.027777777777777776
0.0038461538461538464
0.05555555555555555
0.058823529411764705
0.01834862385321101
0.09090909090909091
0.02702702702702703
0.022222222222222223
0.043478260869565216
0.045454545454545456
0.02
0.029850746268656716
0.1
0.125
0.02631578947368421
0.006191950464396285
0.05555555555555555
0.09090909090909091
0.05
0.011299435028248588
0.002551020408163265
0.043478260869565216
0.03333333333333333
0.1
0.058823529411764705
0.1111111111111111
0.14285714285714285
0.030303030303030304
0.15384615384615385
0.08333333333333333
0.05263157894736842
0.04
0.04
0.009771986970684038
0.0196078431372549
0.1111111111111111
0.024390243902439025
0.1111111111111111
0.038461538461538464
0.005084745762711864
0.1
0.03333333333333333
0.1
0.0022222222222222222
0.0024752475247524753
0.0024691358024691358
0.002564102564102564
0.06896551724137931
0.11764705882352941
0.011111111111111112
0.03333333333333333
0.024390243902439025
0.055555555555

0.10714285714285714
0.125
0.011764705882352941
0.005376344086021506
0.009708737864077669
0.014492753623188406
0.0071174377224199285
0.022727272727272728
0.004777070063694267
0.0032948929159802307
0.003316749585406302
0.022727272727272728
0.00784313725490196
0.0625
0.0032
0.006379585326953748
0.00558659217877095
0.03333333333333333
0.007575757575757576
0.004942339373970346
0.125
0.004901960784313725
0.009009009009009009
0.014084507042253521
0.00847457627118644
0.0125
0.004651162790697674
0.034482758620689655
0.06666666666666667
0.03225806451612903
0.008264462809917356
0.001589825119236884
0.0016181229773462784
0.0032626427406199023
0.004830917874396135
0.0064
0.0016207455429497568
0.003194888178913738
0.001644736842105263
0.0016286644951140066
0.02702702702702703
0.0015625
0.047619047619047616
0.02040816326530612
0.003194888178913738
0.003194888178913738
0.003194888178913738
0.0016474464579901153
0.004830917874396135
0.0016129032258064516
0.001589825119236884
0.0016260162601626016
0.001